# Genetic Algorithms
## Evolutionary Algorithms for Optimization

In [1]:
import numpy as np
import scipy.stats as stt

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

# to use plotly offline, need to initialize with a plot\n",
plyoff.init_notebook_mode(connected=True)
init = go.Figure(data=[go.Scatter({'x':[1, 2], 'y':[42, 42], 'mode':'markers'})], layout=go.Layout(title='Init', xaxis={'title':'x'}, height=100, width=100))
plyoff.iplot(init)

## Mathematical Optimization
Optimization is the process of finding some numerical values that generate a minimum or maximum value for a specified function. Examples include finding the best parameters for a statistical distribution fit to some data, numerically solving differential equations, or feature selection in machine learning.

There are many types and classes of optimization algorithms. Most that have been invented by mathematicianas and computer scientists rely on function derivatives / gradients. Anybody who's studied the topic even slightly will likely remember [Newton-Raphson](https://en.wikipedia.org/wiki/Newton%27s_method) or the [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) - the latter of which even need the second derivaties (the [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix)). Mathematical optimization algorithms typically iterate over two steps:

1.  evaluate current solution
2. find new solution to try

A series of individual solutions are identified and evluated, until the sequence converges to a solution.

Gradient-following methods can have good properties, but also two major issues.

1. The first is the need to compute derivatives. This can often be very difficult - even assuming derivatives can be analytically solved - and time consuming. Furthermore, not all functions we wish to optimizte even have a derivative. What is the derivative of an accuracy loss function for a [Decision Tree Classifier](https://en.wikipedia.org/wiki/Decision_tree_learning)?

2. The second issue is that gradient-followers can easily get stuck in local optima, rather than finding a global optimum point. Most such algorithms rely on being provided an initial values, at which point the necessary derivatives are computed to determine the direction (and perhaps distance) to find the next point to evaluate. Depending on the curvature of the function to optimize, and the initial solution, the algorithm may get stuck in a local, not global, optimum.

The figure generated below demonstrates this second point. The curve to be maximized has two optima with a saddle point in between. Four possible initial values are shown. When the curve's gradient is evaluated at the two $x_0$ points in red, an optimization algorithm will converge to the local optimum. The two in greed will converge to the global optimum. In general, any initial value on the right side of the black vertical line will cause an optimization algorithm to converge to the local optimum; initial values to the left of it will converge to the global optimum.

In [2]:
# get the data
np.random.seed(42)
X1 = stt.norm(loc=0, scale=1).rvs(500)
X2 = stt.norm(loc=2, scale=0.5).rvs(500)
data = np.r_[X1, X2]

# prep for kde
mn, mx = min(data), max(data)
x= np.linspace(mn, mx, 200)

# kde
kde = stt.gaussian_kde(data, bw_method=0.25)
y = kde(x)

# create the annotation
x1 = -1
y1 = kde(x1)[0]
ann1 = dict(x=x1, y=y1, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient →; Local Optimum}$'%x1, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff0000", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#ff0000")
x2 = 0.40
y2 = kde(x2)[0]
ann2 = dict(x=x2, y=y2, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient ←; Local Optimum}$'%x2, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff0000", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#ff0000")
x3 = 1.5
y3 = kde(x3)[0]
ann3 = dict(x=x3, y=y3, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient →; Global Optimum}$'%x3, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#00FF00", opacity=0.8,
            font={'color':'#000000'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#00FF00")
x4 = 3.0
y4 = kde(x4)[0]
ann4 = dict(x=x4, y=y4, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient ; Global Optimum}$'%x4, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#00FF00", opacity=0.8,
            font={'color':'#000000'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#00FF00")
x5 = 0.78698325
y5 = kde(x5)[0]
ann5 = dict(x=x5, y=y5, xref='x1', yref='y1', text='← Fails; Succeeds →', showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#6d72f1", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#636363")

# plot
trcs = [go.Scatter(x=x, y=y, mode='lines', name='Kernel Density Estimate'),
        go.Scatter(x=[x5, x5], y=[min(y), max(y)], mode='lines', line={'color':'black'})]
fig = go.Figure(data=trcs, layout=go.Layout(title='Shortfall of Gradient Following'))
anns = list(fig['layout']['annotations'])
anns.extend([ann1, ann2, ann3, ann4, ann5])
fig.update_layout( annotations=anns)
#plyoff.plot(fig, include_mathjax='cdn')
plyoff.iplot(fig)

## Non-Gradent Following Optimization Algorithms

There are several types of mathematical optimization algorithms which similarly operate on a sequence of individual solutions, but don't use derivatives. I am familiar with [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) and [Golden Section Search](https://en.wikipedia.org/wiki/Golden-section_search) opimization.

## Evolutionary Algorithms
The term [Evolutionary Algorithm (EA)](https://en.wikipedia.org/wiki/Evolutionary_algorithm) refers to a population-based metaheuristic optimization algorithm, and is a subset of evolutionary computation. Broadly, evolutionary algorithms are designed around concepts which come from biological evolution. There are several classes of evolutionary algorithms:

- Differential Evolution
- Evolutionary Programming
- Evolutionary Strategy
- Genetic Algorithm
- Genetic Programming ([see here](https://github.com/ahowe42/baseball))
- Learning Classifier System
- Neuroevolution

This set of lectures is focused on the Genetic Algorithm (GA), but could potentially be extended to include [Genetic Programming](https://en.wikipedia.org/wiki/Genetic_programming).

## Other Gradient Eschewing Algorithms
In addition to EA's, there are several other types of metaheuristic optimization algorithms that are based on the idea of optimizing with a population of potential solutions. These include:

- [Ant Colony Optimization - or Traveling Ant Colony Optimization (TACO)](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms)
- [Particle Swarm Optimization](https://en.wikipedia.org/wiki/Particle_swarm_optimization)
- [Bees Algorithm](https://en.wikipedia.org/wiki/Bees_algorithm)
- [Adaptive Dimensional Search](https://en.wikipedia.org/wiki/Adaptive_dimensional_search)
- [Gaussian Adaptation](https://en.wikipedia.org/wiki/Gaussian_adaptation)
- Harmony Search - special case of [Evolution Strategy](https://en.wikipedia.org/wiki/Evolution_strategy)

I am only familiar with a few of these.

## Genetic Algorithm
The genetic algorithm (GA) is a stochastic population-based metaheuristic optimization algorithm, that borrows concepts from biological evolution. In the GA, a solution to an optimization problem is represented as a binary string of length `n`. In the GA parlance, this is metaphorically called an *individual*, or *chromosome*. Each solution is part of an ensemble of solutions which are considered together; this is called the *population*. The function to be optimized is called the *objective function*, or the *fitness function* - the latter is a metaphor to the biological concept of "survival of the fittest".

Starting from an initial population of `P` individuals, the GA iterates over a few steps:

1. score the fitness of each individuals in the population
2. rank and select each individuals for mating
3. mate pairs of individuals to create a new population
4. apply any GA operators

Each population of individuals is called a *generation*. There are several ways to rank and select individuals for mating. There are also several operators that can be used in the GA, mostly related to creating a new generation.

The semantic meaning of the binary string as a solution to optimizing an objective function (an individual being the most fit in it's environment) depends on the context. For example, if the GA is being used to optimize a real-valued function, the string could be a digital representation of possible real values. The binary string could represent selection flags for a feature selection problem.

In [this research article](https://www.researchgate.net/publication/301770005_Regularized_SVM_Classification_with_a_new_Complexity-Driven_Stochastic_Optimizer), I used the GA to simultaneously select a subset of features and one of 9 kernel functions. The same binary string had one portion interpreted as binary flags, and the other portion as digital representations of the numbers 1 to 9.

In [3]:
# function to print individuals
printIndiv = lambda x: ''.join(['%d'%i for i in x])

In [4]:
''' generate a sample population with scores '''
# generate the population and scores
n = 10
P = 8
np.random.seed(1906)
population = np.random.rand(P, n) > 0.5
fitness = np.random.rand(P)

# sort
stdIndex = np.argsort(fitness)[::-1]
population = population[stdIndex]
fitness = fitness[stdIndex]

# talk
print('Sample population with %d %d-length individuals'%(P,n))
for indx, (score, individual) in enumerate(zip(fitness, population)):
    print('%d: %s = %0.3f'%(indx, printIndiv(individual), score))

Sample population with 8 10-length individuals
0: 1001010001 = 0.889
1: 1000100000 = 0.683
2: 0111101011 = 0.537
3: 0110111110 = 0.448
4: 0101111001 = 0.423
5: 1111011110 = 0.259
6: 1111111110 = 0.156
7: 0001011111 = 0.127


### Mating Ranking & Selection
In the GA, mating is the process of combining two individuals to generate new individuals. I generally have two *parent* solutions mate to create a pair of *child* solutions. I do this is to keep a constant population size, but it's not required. For example, mating could choose to create, based on a randomized choice either:

- a single offspring + new random individual
- a pair of offspring

The new random individual in the first option could be thought of as *adoption*.

I have generally used two approaches to mating ranking & selection.

#### Sorted
In biology, it is generally the case that individuals that are similarly fit for their environment mate together. The metaphor generally holds for the GA, in that solutions with similar objective function scores are mated. This doesn't have to be the case. For example, we could mate pairs of solutions best-fitness to worst-fitness. This approach may help better explore the solution space, but I've never used it. 

This approach to mate ranking and selection is simple, and has the property that each individual mates exactly and only one time. For this, all individuals are sorted according to their fitness scores, then paired off in sequences, so there are $P/2$ mating pairs generated.

#### Roulette
In biology, it is generally the case that the most fit individuals mate the most, thus propagating their successful genes. This metaphor holds by design with the roulette method for generating mating pairs.

The roulette method starts by sorting all individuals according to their fitness scores, then generating a biased roulette bar, in which the individual bins are of gradually decreasing size as computed here and visualized for $P=4$ below:
\begin{equation}
b_i = \frac{2i}{n\left(n+1\right)}, i=1,\ldots,P\text{.}
\end{equation}

<center><img src='./roulette_selection.png' width='400' height='200'></center>

When the cumulative sum of these bin widths is computed, we get upper bounds for the roulette bins, which completely partition the $[0, 1]$ interval. Each bin corresponds to an individual in the population; since the population was already sorted by fitness score, the wider bins correspond to the most fit individuals.

Since we don't want the population to grow, we will generate $P/2$ mating pairs. To do so, $P$ random numbers are generated uniformly from $[0, 1]$ and placed in the appropriate bin. For each random variate in the $i^\text{th}$ bin, the corresponding individual will be selected to mate. In this way, individuals with a better fitness score are overrepresented in the mating pool. The last step specific to this
method is to randomly permute the ordering of the individuals in the mating pool.

Note that this means that, while the most fit individuals will tend to mate the most frequently, they won't just mate with individuals with similar fitness scores.

In [5]:
''' pair individuals for mating - sorted method '''
# skipping sorting, as the population is already sorted
pairs = np.reshape(range(P), (P//2, 2))
# talk
for (indx, pair) in enumerate(pairs):
    print('Mating pair %d: %s (%0.3f) <-> %s (%0.3f)'%(indx, printIndiv(population[pair[0]]), fitness[pair[0]],
                                                       printIndiv(population[pair[1]]), fitness[pair[1]]))

Mating pair 0: 1001010001 (0.889) <-> 1000100000 (0.683)
Mating pair 1: 0111101011 (0.537) <-> 0110111110 (0.448)
Mating pair 2: 0101111001 (0.423) <-> 1111011110 (0.259)
Mating pair 3: 1111111110 (0.156) <-> 0001011111 (0.127)


In [6]:
''' pair individuals for mating - roulette method '''
# generate the bounds and show the bins
binUBounds = np.cumsum(2*np.linspace(P, 1, P)/(P*(P + 1.0)))
bnds = [0] + binUBounds.tolist()
print('Roulette bins: %r'%[('%0.2f'%f, '%0.2f'%t) for f, t in zip(bnds[:-1], bnds[1:])])

# generate mating frequencies
np.random.seed(2022)
rands_in_bins = np.repeat(np.random.rand(P), P) >= np.tile(binUBounds, P)
pairs = np.reshape(np.random.permutation(np.sum(np.reshape(rands_in_bins, [P]*2), axis=1)), (P//2, 2))

# talk
for (indx, pair) in enumerate(pairs):
    print('Mating pair %d: %s (%0.3f) <-> %s (%0.3f)'%(indx, printIndiv(population[pair[0]]), fitness[pair[0]],
                                                       printIndiv(population[pair[1]]), fitness[pair[1]]))

Roulette bins: [('0.00', '0.22'), ('0.22', '0.42'), ('0.42', '0.58'), ('0.58', '0.72'), ('0.72', '0.83'), ('0.83', '0.92'), ('0.92', '0.97'), ('0.97', '1.00')]
Mating pair 0: 0111101011 (0.537) <-> 0110111110 (0.448)
Mating pair 1: 1001010001 (0.889) <-> 0111101011 (0.537)
Mating pair 2: 1111011110 (0.259) <-> 1001010001 (0.889)
Mating pair 3: 0110111110 (0.448) <-> 1001010001 (0.889)


### Crossover
In biology, when a pair of individuals mate, their chromosomes are combined in a process called [chromosomal crossover](https://en.wikipedia.org/wiki/Chromosomal_crossover). An illustration from 1916 by researcher Thomus Hunt Morgan demonstrates the creation of recombinant genes through this process.

<img src='https://upload.wikimedia.org/wikipedia/commons/0/0e/Morgan_crossover_1.jpg' height='500' width='500'>

In the GA, after mating pairs are generated from a population, whether or not the crossover operation is employed is controlled by flipping a coin, biased according to the crossover probability $P_X$. If a random variate generated uniformly on the $[0, 1]$ interval is less than $P_X$, crossover is used. Otherwise, a mating pair produces a pair of offspring that are genetic replicants. Since crossover increases exploration of the objective function's solution space, I generally use a high probability - at least $P_X = 0.7$. I use three types of crossover.

#### Single-Point
In single-point crossover, a single crossover point (see what we did there?) is determined by generating a random value uniformly from the $[1, n]$ interval. The two individuals in a mating pair have their chromosomes traded at that point, as shown in the illustration above.

#### Dual-Point
Dual-point crossover works similarly, except two crossover points are generated randomly in the same was as single-point crossover. The chromosomes are partitioned and traded at these points. This is shown in the illustration below, also from Thomas Hung Morgan.

<img src='https://upload.wikimedia.org/wikipedia/commons/4/45/Morgan_crossover_2.jpg' width='500' height='500'>

#### Uniform
Uniform crossover is the logical endpoint of the sequence of types of crossovers shown here. With uniform crossover, a random variate generated uniformly on the $[0, 1]$ interval is generated for each of the $P$ elements in the chromosomes being crossed over. For each element such that the random value is less than $P_X$, that element of the binary string is traded between the two individuals in the mating pair.

In [14]:
''' single-point crossover '''
# generate the crossover flags
np.random.seed(42)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoint = 0
        n1, n2 = p1, p2
    else:
        # crossover
        xoverpoint = np.random.randint(1, n-1)
        n1 = np.concatenate((p1[:xoverpoint], p2[xoverpoint:]))
        n2 = np.concatenate((p2[:xoverpoint], p1[xoverpoint:]))
    # talk
    print('Mating pair %d (xover=%d): %s <-> %s --> %s & %s'%(indx, xoverpoint, printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))

Mating pair 0 (xover=0): 0111101011 <-> 0110111110 --> 0111101011 & 0110111110
Mating pair 1 (xover=7): 1001010001 <-> 0111101011 --> 1001010011 & 0111101001
Mating pair 2 (xover=2): 1111011110 <-> 1001010001 --> 1101010001 & 1011011110
Mating pair 3 (xover=0): 0110111110 <-> 1001010001 --> 0110111110 & 1001010001


In [13]:
''' dual-point crossover '''
# generate the crossover flags
np.random.seed(42)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoints = [0,0]
        n1, n2 = p1, p2
    else:
        # crossover
        xoverpoints = np.sort(np.random.permutation(n-1)[:2]+1)
        n1 = np.concatenate((p1[:xoverpoints[0]], p2[xoverpoints[0]:xoverpoints[1]], p1[xoverpoints[1]:]))
        n2 = np.concatenate((p2[:xoverpoints[0]], p1[xoverpoints[0]:xoverpoints[1]], p2[xoverpoints[1]:]))
    # talk
    print('Mating pair %d (xover=[%d, %d]): %s <-> %s --> %s & %s'%(indx, xoverpoints[0], xoverpoints[1], printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))

Mating pair 0 (xover=[0, 0]): 0111101011 <-> 0110111110 --> 0111101011 & 0110111110
Mating pair 1 (xover=[5, 8]): 1001010001 <-> 0111101011 --> 1001001001 & 0111110011
Mating pair 2 (xover=[1, 7]): 1111011110 <-> 1001010001 --> 1001010110 & 1111011001
Mating pair 3 (xover=[0, 0]): 0110111110 <-> 1001010001 --> 0110111110 & 1001010001


In [19]:
''' uniform-point crossover '''
# generate the crossover flags
np.random.seed(1211)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoint = [0]*n
        n1, n2 = p1, p2
    else:
        # crossover
        xoverpoints = probXover > np.random.rand(n)
        n1 = p1*xoverpoints + p2*~xoverpoints
        n2 = p1*~xoverpoints + p2*xoverpoints
    # talk
    print('Mating pair %d (xover=%s): %s <-> %s --> %s & %s'%(indx, printIndiv(xoverpoints), printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))

Mating pair 0 (xover=1101100101): 0111101011 <-> 0110111110 --> 0111111011 & 0110101110
Mating pair 1 (xover=0011110010): 1001010001 <-> 0111101011 --> 0101011001 & 1011100011
Mating pair 2 (xover=0011110010): 1111011110 <-> 1001010001 --> 1111011110 & 1001010001
Mating pair 3 (xover=1011011011): 0110111110 <-> 1001010001 --> 0010011010 & 1101110101


### Mutation
After mating pairs use crossover to generate a new population of individuals, the new individuals' chromosomes are mutated.

### GA Engineering

### Elitism

In [ ]:
exploitation and exploration